# 海上风力发电场

## 目标和前提条件

在本示例中，您将学习如何解决海上风力发电问题。该问题的目标是确定应该如何铺设水下电缆以最低成本连接海上风电场电网。我们将向您展示如何使用Gurobi Python API构建该问题的混合整数规划(MIP)模型，然后使用Gurobi优化器找到问题的最优解。

这个建模示例属于初级水平，我们假设您了解Python并对构建数学优化模型有一定了解。

**下载代码库** <br />
您可以通过点击[此处](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

## 背景动机

全球气候变化已经对环境产生了可观察到的影响。冰川已经缩小，河流和湖泊的冰层比预期更早破碎，植物和动物物种受到影响，树木开花时间也早于预期。全球气候变化的潜在未来影响包括更频繁的野火、部分地区更长时间的干旱以及热带风暴的数量、持续时间和强度的增加。[1]

气候变化减缓包括限制全球变暖规模或速度及其相关影响的行动。
气候变化减缓的首要挑战是消除煤炭、石油，最终还包括天然气的燃烧。这可能是最艰巨的挑战，因为富裕国家的居民在日常生活的方方面面都依赖化石燃料制成的产品。此外，发展中国家的公民也想要并且可以说理应享有同样的舒适。减少对化石燃料依赖没有完美的解决方案（例如，碳中和生物燃料可能推高食品价格并导致森林破坏，而核能虽然不排放温室气体，但会产生放射性废物）。其他替代方案包括植物衍生塑料、生物柴油和风能。[2]

海上风力发电是指在水体中（通常是在海洋中）建造风电场，利用风能发电。与陆地相比，海上有更高的风速，因此海上风力发电的发电量相对于装机容量更高。

将风力涡轮机设置在海上的优势在于海上风力更强，而且与大陆上空的风不同，海上的微风在下午也可能很强，这与人们用电高峰时段相匹配。海上涡轮机还可以靠近沿海的负荷中心（如大城市），消除了新建远距离输电线路的需求。


## 问题描述

海上风电场是在海上放置的风力涡轮机群，利用强劲的海上风力。这些强风可以产生更多的电力，但海上风电场的安装和运营成本比陆上风电场更高。

我们将使用MIP模型来降低建设海上风电场的部分成本。我们将计算如何铺设连接涡轮机的水下电缆。这些电缆对于将涡轮机产生的电力输送到陆地是必要的。我们计算的方案将最小化水下电缆的安装成本，同时确保每个涡轮机都连接到岸上，且每条电缆都有足够的容量处理产生的电流。

在我们的示例中，一个风电场正在丹麦西海岸建设。海岸上有一个发电站，所有电力必须传输到那里以分配到电网。风电场中还有传输站，可以收集多个涡轮机的电力并通过单根电缆传输到岸上。

安装电缆时我们必须考虑两个因素。首先，在海底铺设电缆有固定成本。这个成本与连接两个站点之间的距离成正比。其次，我们必须考虑电缆中流动的电流大小。承载大电流的连接需要粗电缆。粗电缆比细电缆更贵。

这个优化问题的目标是决定应该铺设哪些电缆以最小成本连接风电场电网。

海上风电场优化问题的模型是一种更一般的优化模型的实例，称为固定费用网络流问题。固定费用网络流问题可以应用于大量的商业问题 - 例如，在通信和运输网络的规划中。



## 解决方案方法

数学规划是一种声明式方法，模型构建者通过构建数学优化模型来捕捉复杂决策问题的关键方面。Gurobi优化器使用最先进的数学和计算机科学来解决这些模型。

一个数学优化模型有五个组成部分，即：

* 集合和索引。
* 参数。
* 决策变量。
* 目标函数。
* 约束条件。

现在我们为海上风电场问题提出一个MIP模型。

## 模型公式

### 集合和索引

$G(V,E)$: 表示风电场网络的图，其中$V$是顶点集，$E$是边集。涡轮机、传输站和发电站是图中顶点集$V$中的顶点。潜在电缆的集合是图的边。

### 参数

$s_{i} \in \mathbb{R}$: 顶点$i \in V$处的电力供应。由于涡轮机供应电力，它们是源顶点，$s_{i} > 0$。传输站不向网络供应或移除电力，因此它们的$s_{i} = 0$。海岸上的发电站是一个汇点，移除风电场的所有电力，因此它的$s_{i} < 0$。

$u_{i,j} \in \mathbb{R}^+ $: 从顶点$i \in V$到顶点$j \in V$的电缆可以承受的最大电流容量。

$c_{i,j} \in \mathbb{R}^+$: 从顶点$i \in V$到顶点$j \in V$的单位电流流动成本，即我们必须支付的增加电缆厚度以处理电流增加的价格。

$f_{i,j} \in \mathbb{R}^+$: 从顶点$i \in V$到顶点$j \in V$铺设电缆的固定成本，是顶点之间距离与每英里的成本的乘积。

### 决策变量

$install_{i,j} \in \{0, 1 \}$: 如果我们从顶点$i \in V$到顶点$j \in V$铺设电缆，则该变量等于1；否则为0。

$flow_{i,j} \geq 0$: 这个非负连续变量表示从顶点$i \in V$到顶点$j \in V$的电流流量。

这个优化模型的目标是决定图中哪些潜在边应该以最低成本使用。

### 目标函数

- **总成本**。我们希望最小化安装电缆的总成本。左边的项是可变成本（即根据电缆中的电流变化的成本）。右边的项是安装电缆的固定成本。

\begin{equation}
\text{Max} \quad Z = \sum_{(i,j) \in E} c_{i,j} \cdot flow_{i,j} + \sum_{(i,j) \in E} f_{i,j} \cdot install_{i,j} 
\tag{0}
\end{equation}

### 约束条件

- **平衡**。对于每个顶点$i \in V$，我们希望确保网络中的电流守恒。

\begin{equation}
\sum_{j:(i,j) \in E} flow_{i,j} - \sum_{j:(j,i) \in E} flow_{j,i} = s_{i} \quad \forall i \in V
\tag{1}
\end{equation}

- **容量**。对于每条边$(i,j) \in E$，我们希望强制执行每条电缆的最大电流容量限制。

\begin{equation}
0 \leq flow_{i,j} \leq u_{i,j} \cdot install_{i,j}  \quad \forall (i,j) \in E
\tag{2}
\end{equation}

## Python实现

本示例考虑了三个涡轮机、一个传输站和两个发电站。风电场网络中每个顶点的电流流出量如下表所示。请记住，由于涡轮机供应电力，它们的容量为正。传输站不向网络供应或移除电力，因此它们的容量为零。海岸上的发电站是汇点，移除风电场网络的所有电力，因此它们有电力需求，在这种情况下我们使用负数表示。

| <i></i> | 容量（MW） |  
| --- | --- | 
| 顶点 1 | 4 |
| 顶点 2 | 3 |
| 顶点 3 | 2 |
| 顶点 4 | 0 |
| 顶点 5 | -6 |
| 顶点 6 | -3 |


风电场网络中每条边的容量、流动成本和固定成本如下表所示。

| <i></i> | 容量（MW）  | 流动成本（百万欧元） | 固定成本（百万欧元）| 
| --- | --- | --- | --- |
| 边: (0,4) | 4 | 1 | 1 |
| 边: (0,3) | 2 | 1 | 1 |
| 边: (1,3) | 3 | 1 | 1 |
| 边: (2,5) | 2 | 1 | 1 |
| 边: (3,4) | 2 | 1 | 1 |
| 边: (3,5) | 1 | 1 | 1 |


现在我们导入Gurobi Python模块。然后，我们使用给定的数据初始化数据结构。

In [ ]:
# %pip install gurobipy

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# 参数

vertices = {0: 4, 1: 3, 2: 2, 3: 0, 4: -6, 5: -3}
edges, cap, flow_cost, fixed_cost = gp.multidict({
    (0,4): [4,1,1],
    (0,3): [2,1,1],
    (1,3): [3,1,1],
    (2,5): [2,1,1],
    (3,4): [2,1,1],
    (3,5): [1,1,1]
})

### 模型部署

现在我们通过定义决策变量、约束条件和目标函数来确定海上风电场问题的MIP模型。接下来，我们开始优化过程，Gurobi找到在海上风电场网络中铺设电缆的计划，以最小化总成本。

In [ ]:
# MIP 模型公式

m = gp.Model("offshore_wind_farming")

# 添加变量
install = m.addVars(edges, vtype=GRB.BINARY, name="Install")
flow = m.addVars(edges, vtype=GRB.CONTINUOUS, name="Flow")

# 添加约束条件
m.addConstrs((flow.sum(v,'*') - flow.sum('*',v) == supply for v, supply in vertices.items()), name="Flow_conservation")
m.addConstrs((flow[e] <= cap[e]*install[e] for e in edges), name="Install2flow")

# 设置目标函数
m.setObjective(flow.prod(flow_cost) + install.prod(fixed_cost), GRB.MINIMIZE)

m.optimize()

Using license file c:\gurobi\gurobi.lic
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 12 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x03b3989f
Variable types: 6 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 6e+00]
Presolve removed 12 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 17 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.700000000000e+01, best bound 1.700000000000e+01, gap 0.0000%


## 分析


优化模型的结果显示，最小总成本值为1700万欧元。让我们看看实现该最优结果的解决方案。

### 电缆安装计划

该计划确定了海上风电场网络中电缆的布局。

In [ ]:
# 显示计划在海上风电场网络中安装的边

for origin, end in install.keys():
    if (abs(install[origin, end].x) > 0.5):
        print(f"Install cable from location {origin + 1} to location {end + 1} in the offshore wind farming network ")
        


Install cable from location 1 to location 5 in the offshore wind farming network 
Install cable from location 2 to location 4 in the offshore wind farming network 
Install cable from location 3 to location 6 in the offshore wind farming network 
Install cable from location 4 to location 5 in the offshore wind farming network 
Install cable from location 4 to location 6 in the offshore wind farming network 


### 电缆容量计划

该计划确定了每条已安装电缆的电流流量容量（MW）。

In [ ]:
# 每条已安装电缆的当前流量容量

for origin, end in flow.keys():
    if (abs(flow[origin, end].x) > 1e-6):
         print(f"The capacity of cable installed from location {origin + 1} to location {end + 1} is {flow[origin, end].x} MW ")

The capacity of cable installed from location 1 to location 5 is 4.0 MW 
The capacity of cable installed from location 2 to location 4 is 3.0 MW 
The capacity of cable installed from location 3 to location 6 is 2.0 MW 
The capacity of cable installed from location 4 to location 5 is 2.0 MW 
The capacity of cable installed from location 4 to location 6 is 1.0 MW 


## 结论

在本示例中，我们解决了一个海上风电场问题，我们希望最小化铺设水下电缆的成本，以收集海上风电场网络产生的电力。我们学习了如何将问题构建为MIP模型。此外，我们学习了如何实现MIP模型公式并使用Gurobi Python API解决它。

## 参考文献

[1] https://climate.nasa.gov/effects/

[2] https://www.scientificamerican.com/article/10-solutions-for-climate-change/

版权所有 © 2020 Gurobi Optimization, LLC